<a href="https://colab.research.google.com/github/Romasha-Guin/Final-Year-Project/blob/main/Real_Time_Speech_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install ipywidgets
!apt-get install -y portaudio19-dev
!pip install pyaudio
!pip install vosk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
portaudio19-dev is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [12]:
import ipywidgets as widgets
from IPython.display import display, Audio, HTML, clear_output
from queue import Queue
from threading import Thread
import subprocess
import json
import time
import pyaudio
from vosk import Model, KaldiRecognizer
import asyncio

In [13]:
messages = Queue()
recordings = Queue()

In [14]:
record_button = widgets.Button(
    description='Record',
    disabled=False,
    button_style='success',
    tooltip='Record',
    icon='microphone'
)

In [15]:
stop_button = widgets.Button(
    description='Stop',
    disabled=False,
    button_style='warning',
    tooltip='Stop',
    icon='stop'
)

In [16]:
output = widgets.Output()

In [17]:
def start_recording(data):
    loop = asyncio.get_event_loop()
    messages.put(True)

    with output:
        clear_output(wait=True)
        display("Starting...")

        record = loop.run_in_executor(None, record_microphone_async)
        transcribe = loop.run_in_executor(None, speech_recognition, output)

        try:
            loop.run_until_complete(asyncio.gather(record, transcribe))
        except Exception as e:
            display(f"Error: {str(e)}")

In [18]:
def stop_recording(data):
    with output:
        messages.get()
        display("Stopped.")

In [19]:
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output)

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle(), tooltip='Record')

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle(), tooltip='Stop')

Output()

In [20]:
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 20
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

In [21]:
def record_microphone_async(chunk=1024):
    p = pyaudio.PyAudio()

    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate=FRAME_RATE,
                    input=True,
                    input_device_index=2,
                    frames_per_buffer=chunk)

    frames = []

    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames = []

    stream.stop_stream()
    stream.close()
    p.terminate()

In [22]:
model = Model(model_name="vosk-model-en-us-0.22")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

vosk-model-en-us-0.22.zip: 100%|██████████| 1.78G/1.78G [01:43<00:00, 18.5MB/s]


In [23]:
def speech_recognition(output):
    while not messages.empty():
        frames = recordings.get()

        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)["text"]

        try:
            cased = subprocess.check_output('python recasepunc/recasepunc.py predict recasepunc/checkpoint', shell=True, text=True, input=text)
            output.append_stdout(cased)
        except subprocess.CalledProcessError as e:
            output.append_stderr(f"Error in recasepunc: {str(e)}")

        time.sleep(1)